In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import time
import os
import torch.nn.functional as F
from tqdm import tqdm

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True
1


In [6]:
learning_rate = 0.001
num_epochs = 2
num_workers = 2
batch_size = 100

In [7]:
# Load CIFAR-100 data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


Files already downloaded and verified
Files already downloaded and verified


In [8]:
# Define the Teacher model
class Teacher(nn.Module):
    def __init__(self):
        super(Teacher, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, 5)
        self.fc1 = nn.Linear(128*5*5, 256)
        self.fc2 = nn.Linear(256, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 128*5*5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the Student model
class Student(nn.Module):
    def __init__(self):
        super(Student, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*14*14, 120)
        self.fc2 = nn.Linear(120, 100)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 16*14*14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
# def fairness_constraint(outputs, sensitive_attrs):
#     # This is a simplified fairness constraint for demonstration purposes.
#     group0_indices = (sensitive_attrs < 5).nonzero(as_tuple=True)
#     group1_indices = (sensitive_attrs >= 5).nonzero(as_tuple=True)
#     positive_rate_group0 = (outputs[group0_indices].max(1)[1] >= 5).float().mean()
#     positive_rate_group1 = (outputs[group1_indices].max(1)[1] >= 5).float().mean()
#     return torch.abs(positive_rate_group0 - positive_rate_group1)

# def adversarial_train_model(model, dataloader, criterion, optimizer, epoch):
#     model.train()
#     running_loss = 0.0
#     progress_bar = tqdm(enumerate(dataloader, 0), total=len(dataloader), unit="batch")
#     for i, data in progress_bar:
#         inputs, labels = data
#         sensitive_attrs = labels  # Assuming label itself as a sensitive attribute
#         optimizer.zero_grad()
#         outputs = model(inputs.to(device))
#         loss = criterion(outputs, labels.to(device))
#         fairness_penalty = fairness_constraint(outputs, sensitive_attrs.to(device))
#         combined_loss = loss + fairness_penalty  # Combine original loss and fairness penalty
#         combined_loss.backward()
#         optimizer.step()
#         running_loss += combined_loss.item()
#         progress_bar.set_description(f"Epoch {epoch} Loss: {running_loss/(i+1):.4f}")
#     return running_loss / len(dataloader)

In [9]:
# Define the fair_metric function from the RELIANT code
def fair_metric(pred, labels, sens):
    idx_s0 = sens == 0
    idx_s1 = sens == 1
    idx_s0_y1 = idx_s0 & (labels == 1)
    idx_s1_y1 = idx_s1 & (labels == 1)

    if idx_s0.sum().item() == 0 or idx_s1.sum().item() == 0 or idx_s0_y1.sum().item() == 0 or idx_s1_y1.sum().item() == 0:
        return torch.tensor(0.0), torch.tensor(0.0)  # Return zeros

    # print(idx_s0.sum().item(), idx_s1.sum().item(), idx_s0_y1.sum().item(), idx_s1_y1.sum().item())
    # smoothing term added
    parity = torch.abs((pred[idx_s0].sum() / (idx_s0.sum().item() + 1e-8)) - (pred[idx_s1].sum() / (idx_s1.sum().item() + 1e-8)))
    equality = torch.abs((pred[idx_s0_y1].sum() / (idx_s0_y1.sum().item() + 1e-8)) - (pred[idx_s1_y1].sum() / (idx_s1_y1.sum().item() + 1e-8)))

    return parity.item(), equality.item()

In [10]:
# Define the approx_loss and approx_loss_eo functions from the RELIANT code
def approx_loss(logits, sens, idx):
    g1 = np.argwhere(sens == 0.0).reshape(-1)
    g2 = np.argwhere(sens == 1.0).reshape(-1)
    idx_set = set(idx.cpu().numpy())
    g1 = np.array(list(set(g1) & idx_set))
    g2 = np.array(list(set(g2) & idx_set))

    if g1.shape[0] == 0 or g2.shape[0] == 0:
        return torch.tensor(0.0) # Return zero
    
    # print(g1.shape[0], g2.shape[0])
    # smoothing term added
    loss = torch.square(approx_func(logits[g1]).sum(axis=0) / (g1.shape[0] + 1e-8) - approx_func(logits[g2]).sum(axis=0) / (g2.shape[0] + 1e-8)).sum()
    return loss


In [11]:
def approx_loss_eo(logits, sens, labels, idx):
    g1 = np.argwhere(sens == 0).reshape(-1)
    g2 = np.argwhere(sens == 1).reshape(-1)
    g = np.argwhere(labels.cpu().numpy() == 1).reshape(-1)
    idx_set = set(idx.cpu().numpy())
    g1 = np.array(list(set(g1) & set(g) & idx_set))
    g2 = np.array(list(set(g2) & set(g) & idx_set))
    
    if g1.shape[0] == 0 or g2.shape[0] == 0:
        return torch.tensor(0.0)  # Return zero 
        
    # print(g1.shape[0], g2.shape[0])
    # smoothing term added
    loss = torch.square(approx_func(logits[g1]).sum(axis=0) / (g1.shape[0] + 1e-8) - approx_func(logits[g2]).sum(axis=0) / (g2.shape[0] + 1e-8)).sum()
    return loss

In [12]:
def approx_func(s):
    x = 2 * s - 1
    result = 1 / 2 + 1 / 2 * x - 1 / 8 / 2 * (5 * x * x * x - 3 * x) + 1 / 16 / 8 * (
                63 * x * x * x * x * x - 70 * x * x * x + 15 * x) - 5 / 128 / 16 * (
                       429 * x * x * x * x * x * x * x - 693 * x * x * x * x * x + 315 * x * x * x - 35 * x) + 7 / 256 / 128 * (
                       12155 * x * x * x * x * x * x * x * x * x - 25740 * x * x * x * x * x * x * x + 18018 * x * x * x * x * x - 4620 * x * x * x + 315 * x)
    # print(s, x, result)
    return result

In [13]:
def adversarial_train_model(model, dataloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    progress_bar = tqdm(enumerate(dataloader, 0), total=len(dataloader), unit="batch")
    for i, data in progress_bar:
        inputs, labels = data
        sensitive_attrs = labels  # Assuming label itself as a sensitive attribute
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        
        # Incorporate fairness metrics
        fairness_penalty_sp, fairness_penalty_eo = fair_metric(outputs, labels, sensitive_attrs)
        fairness_penalty_approx = approx_loss(outputs, sensitive_attrs, torch.arange(len(labels)).to(device))
        fairness_penalty_approx_eo = approx_loss_eo(outputs, sensitive_attrs, labels, torch.arange(len(labels)).to(device))
        combined_loss = loss + fairness_penalty_sp + fairness_penalty_eo + fairness_penalty_approx + fairness_penalty_approx_eo

        print(combined_loss.item(), fairness_penalty_sp, fairness_penalty_eo, fairness_penalty_approx.item(), fairness_penalty_approx_eo.item())
        combined_loss.backward()
        optimizer.step()
        running_loss += combined_loss.item()
        progress_bar.set_description(f"Epoch {epoch} Loss: {running_loss/(i+1):.4f}")
    return running_loss / len(dataloader)

In [14]:
# Initialize models, optimizers, and loss function
teacher = Teacher().to(device)
student = Student().to(device)
optimizer_teacher = optim.Adam(teacher.parameters(), lr=learning_rate)
optimizer_student = optim.Adam(student.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [15]:
# Training Loop
for epoch in range(num_epochs):
    train_loss_teacher = adversarial_train_model(teacher, trainloader, criterion, optimizer_teacher, epoch)
    print(f'Teacher Training Loss: {train_loss_teacher}')
    train_loss_student = adversarial_train_model(student, trainloader, criterion, optimizer_student, epoch)
    print(f'Student Training Loss: {train_loss_student}')

Epoch 0 Loss: 4.5875:   0%|▏                                                                      | 1/500 [00:00<03:05,  2.70batch/s]

4.609371185302734 tensor(0.) tensor(0.) 0.0 0.0
4.649016380310059 tensor(0.) tensor(0.) 0.0 0.0
4.565587043762207 tensor(0.) tensor(0.) 0.0 0.0
4.584012985229492 tensor(0.) tensor(0.) 0.0 0.0
4.597846508026123 tensor(0.) tensor(0.) 0.0 0.0
4.599612236022949 tensor(0.) tensor(0.) 0.0 0.0
4.539072036743164 tensor(0.) tensor(0.) 0.0 0.0
4.5558342933654785 tensor(0.) tensor(0.) 0.0 0.0


Epoch 0 Loss: 4.5051:   3%|██▏                                                                   | 16/500 [00:00<00:14, 34.39batch/s]

4.5614423751831055 tensor(0.) tensor(0.) 0.0 0.0
4.469376087188721 tensor(0.) tensor(0.) 0.0 0.0
4.492891311645508 tensor(0.) tensor(0.) 0.0 0.0
4.476677417755127 tensor(0.) tensor(0.) 0.0 0.0
4.524063587188721 tensor(0.) tensor(0.) 0.0 0.0
4.341625690460205 tensor(0.) tensor(0.) 0.0 0.0
4.444364547729492 tensor(0.) tensor(0.) 0.0 0.0
4.4207634925842285 tensor(0.) tensor(0.) 0.0 0.0
4.468938827514648 tensor(0.) tensor(0.) 0.0 0.0
4.394364833831787 tensor(0.) tensor(0.) 0.0 0.0
4.470792770385742 tensor(0.) tensor(0.) 0.0 0.0
4.335714817047119 tensor(0.) tensor(0.) 0.0 0.0
4.362329483032227 tensor(0.) tensor(0.) 0.0 0.0


Epoch 0 Loss: 4.4090:   6%|████                                                                  | 29/500 [00:00<00:09, 48.04batch/s]

...

3.170370578765869 tensor(0.) tensor(0.) 0.0 0.0
3.2705609798431396 tensor(0.) tensor(0.) 0.0 0.0
3.227724552154541 tensor(0.) tensor(0.) 0.0 0.0
2.93156099319458 tensor(0.) tensor(0.) 0.0 0.0
2.6206259727478027 tensor(0.) tensor(0.) 0.0 0.0
3.2306973934173584 tensor(0.) tensor(0.) 0.0 0.0
2.7386345863342285 tensor(0.) tensor(0.) 0.0 0.0
3.107853889465332 tensor(0.) tensor(0.) 0.0 0.0
2.9485559463500977 tensor(0.) tensor(0.) 0.0 0.0
3.1117610931396484 tensor(0.) tensor(0.) 0.0 0.0


Epoch 1 Loss: 3.0669:  99%|████████████████████████████████████████████████████████████████████▎| 495/500 [00:07<00:00, 73.24batch/s]

2.899087905883789 tensor(0.) tensor(0.) 0.0 0.0
2.870023727416992 tensor(0.) tensor(0.) 0.0 0.0
2.8947370052337646 tensor(0.) tensor(0.) 0.0 0.0
2.7515344619750977 tensor(0.) tensor(0.) 0.0 0.0
2.967581272125244 tensor(0.) tensor(0.) 0.0 0.0


Epoch 1 Loss: 3.0660: 100%|█████████████████████████████████████████████████████████████████████| 500/500 [00:07<00:00, 68.31batch/s]

2.957993745803833 tensor(0.) tensor(0.) 0.0 0.0
3.088606834411621 tensor(0.) tensor(0.) 0.0 0.0
3.1044037342071533 tensor(0.) tensor(0.) 0.0 0.0
2.89113712310791 tensor(0.) tensor(0.) 0.0 0.0
2.8429088592529297 tensor(0.) tensor(0.) 0.0 0.0
Student Training Loss: 3.066043072223663


In [13]:
# Evaluation and Metrics
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    start_time = time.time()
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds.cpu().numpy())
    end_time = time.time()
    inference_time = end_time - start_time
    all_labels = np.concatenate(all_labels)
    all_preds = np.concatenate(all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')
    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return inference_time, precision, recall, accuracy, f1


In [14]:
# Get metrics for both teacher and student
metrics_teacher = evaluate(teacher, testloader)
metrics_student = evaluate(student, testloader)
print('Teacher Metrics:', metrics_teacher)
print('Student Metrics:', metrics_student)

Teacher Metrics: (20.571057081222534, 0.6747847588117001, 0.6709, 0.6709, 0.6643685930197498)
Student Metrics: (6.319691181182861, 0.606594311336325, 0.5805, 0.5805, 0.5771152991241887)


In [17]:
# Save the model weights
torch.save(student.state_dict(), 'student.pth')
torch.save(teacher.state_dict(), 'teacher.pth')

In [18]:
# Save the architecture
import json
student_architecture = {
    'conv1': [3, 16, 5],
    'pool': [2, 2],
    'fc1': [16*14*14, 120],
    'fc2': [120, 100]  # original number of classes
}
with open('student_architecture.json', 'w') as f:
    json.dump(student_architecture, f)
